# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [21]:
pip install openai -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [22]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [23]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [24]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [25]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_6917fca1950c8191936b987c0b117850


### Upload Files

In [26]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-1xctpDrfYUaLCjcYoLBDWD


### Attach File to Vector Store

In [27]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-1xctpDrfYUaLCjcYoLBDWD


### Query the Vector Store

In [28]:
query = "the latest development in generativeAI"

In [29]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

## OpenAI Response API

### Simple Response

In [31]:
display(Markdown(simple_response.output_text))

As of the latest developments, generative AI continues to advance rapidly across multiple domains:

1. **Text Generation**: Models like OpenAI's GPT series have become more sophisticated, with improvements in coherence, context understanding, and innovative applications such as detailed content creation, summarization, and translation.

2. **Image and Video Synthesis**: Tools like DALL-E and Midjourney are producing highly detailed and creative images from text prompts, while advancements in video generation are enabling more realistic and dynamic video content.

3. **Music and Audio**: AI models are capable of composing music in various styles and genres, simulating human-like voices, and even creating entirely new sounds.

4. **Code Generation**: AI tools like Copilot are assisting developers by suggesting code snippets, facilitating debugging, and automating routine programming tasks.

5. **Multi-modal Models**: The integration of different types of data (text, image, audio) into unified models is enhancing capabilities for more complex applications and cross-domain creative tasks.

6. **Ethical AI**: There are ongoing efforts to make generative AI more ethical and transparent, with research focusing on reducing biases, improving interpretability, and ensuring responsible deployment.

7. **Accessibility**: Generative AI is becoming more accessible with APIs and user-friendly interfaces, enabling a wider range of users to leverage AI capabilities in various sectors, from education to entertainment.

These developments indicate a trend towards more integrated, efficient, and user-friendly AI applications with a focus on ethical considerations.

### File Search Response

In [32]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [33]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include several key advancements and applications:

1. **Agentic Workflows**: Amazon Web Services is exploring the future of AI with a focus on agentic workflows, which are being showcased alongside startups like NeuralSeek and Tarpit AI.

2. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis tool using generative AI, SAP BTP, and AWS Bedrock to assess risk and boost resilience.

3. **Content Creation**: Generative AI is being used to create cinematic videos from prompts, incorporating audio, physics, and cameos, which is a game-changer for creators.

4. **Market Growth**: The global generative AI market is expected to reach $1.18 billion this year, highlighting its rapid growth and adoption across industries.

5. **Creative Industries**: Generative AI is transforming creative industries by enabling new forms of content generation and design systems.

These developments illustrate the expanding role of generative AI in various sectors, from supply chain management to creative content production.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [34]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [35]:
display(Markdown(web_search_response.output_text))

Here’s an in-depth update on the *latest developments in generative AI* as of today, **November 15, 2025**. I've structured the response across key areas to help you navigate the most significant breakthroughs and industry movements.  

---

Key Developments in Generative AI

1. **OpenAI’s GPT‑5.1 Release**  
   - On **November 12, 2025**, OpenAI released **GPT‑5.1**, a refined version of its GPT‑5 foundation model ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5.1?utm_source=openai)).  
   - GPT‑5.1 introduces enhancements such as reduced hallucinations, improved instruction following, faster performance, and a more “warm” default personality. It includes two operational modes: **Instant**, for quick responses, and **Thinking**, optimized for complex reasoning tasks ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5.1?utm_source=openai)).

2. **Brain‑Inspired "Dragon Hatchling" Architecture by Pathway**  
   - Researchers unveiled a novel architecture termed **Dragon Hatchling**, designed to mimic the dynamic neural structures of the human brain. Unlike traditional transformer-based LLMs, this model continuously adapts its internal structure in real time—forming and adjusting pathways akin to neuroplasticity.  
   - Early tests showed performance comparable to GPT‑2, signaling promise for models that can learn and generalize over time without full retraining ([livescience.com](https://www.livescience.com/technology/artificial-intelligence/new-dragon-hatchling-ai-architecture-modeled-after-the-human-brain-could-be-a-key-step-toward-agi-researchers-claim?utm_source=openai)).

3. **World Generation via Spatial AI by World Labs**  
   - **World Labs**, co-founded by Fei‑Fei Li, introduced **Marble**, a generative AI tool for creating 3D worlds from text, image, or video prompts. Launched in November 2025, Marble integrates with Unity and Unreal Engine, offering creators a streamlined environment-creation workflow ([theverge.com](https://www.theverge.com/ai-artificial-intelligence/820016/world-labs-is-betting-on-world-generation-as-the-next-ai-frontier?utm_source=openai)).  
   - The platform includes tiered pricing—starting with a free version and scaling to a $95/month “Max” tier—for commercial and advanced usage ([theverge.com](https://www.theverge.com/ai-artificial-intelligence/820016/world-labs-is-betting-on-world-generation-as-the-next-ai-frontier?utm_source=openai)).

4. **DeepMind Solves Fluid Dynamics Singularity Problem**  
   - In a breakthrough bridging generative AI and scientific research, **DeepMind** applied tailored AI models to unravel a century-old fluid dynamics problem: identifying previously unknown families of **unstable singularities**. This has wide implications for understanding turbulence, with benefits in engineering, meteorology, and medicine ([businessinsider.com](https://www.businessinsider.com/google-deepmind-cracks-century-old-physics-mystery-ai-fluid-dynamics-2025-11?utm_source=openai)).

5. **Google’s Nano Banana 2 Leak – Next-Gen Image Generator**  
   - A leak hints at Google developing **Nano Banana 2**, a potential upgrade to its image generation tools, offering **native 2K image generation**, **4K upscaling**, and a multi-step refinement process called **“Lightbox”** to enhance realism and visual coherence ([tomsguide.com](https://www.tomsguide.com/ai/ai-image-video/nano-banana-2-leak-reveals-googles-new-studio-lightbox-heres-what-we-know?utm_source=openai)).  
   - Though unconfirmed, this lines up with Google’s ongoing efforts to lead in AI-based creative tools ([tomsguide.com](https://www.tomsguide.com/ai/ai-image-video/nano-banana-2-leak-reveals-googles-new-studio-lightbox-heres-what-we-know?utm_source=openai)).

---

6. **Industry & Infrastructure Movements**

   - **AWS & Anthropic Partnership**: Amazon Web Services is investing up to $4 billion in Anthropic—both through cloud credits and equity—making AWS its primary cloud provider. This partnership grants Anthropic access to AWS’s Trainium and Inferentia accelerators and paves the way for Claude models on AWS Bedrock ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-latest-breakthroughs-and-developments?utm_source=openai)).
   
   - **Oracle’s Generative AI Offering**: Oracle unveiled a beta of **OCI Generative AI**, offering Cohere-powered models (e.g., Command, Summarization, Embeddings) fully hosted on Oracle Cloud Infrastructure. This integration targets enterprise use cases in chatbots, summarization, and search, with predictable pricing and deployment security ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-latest-breakthroughs-and-developments?utm_source=openai)).

---

Implications and Outlook

- **Adaptive and Agentic Intelligence**: Models like **Dragon Hatchling** move us toward agents capable of real-time adaptation and lifelong learning—key steps toward artificial general intelligence (AGI).  
- **Multimodal and Spatial Creativity**: Tools such as **Marble** emphasize the expansion beyond text and image—into immersive 3D worlds—powering innovation in game development, architecture, media, and simulation.  
- **Scientific & Technical Leap**: DeepMind’s singularity discovery illustrates the ability of generative AI to tackle complex scientific challenges, positioning AI as a valid co-researcher rather than just a creative tool.  
- **Enterprise Acceleration**: Major cloud players (AWS, Oracle) are embedding generative AI into infrastructure, underscoring enterprise demand for scalable, secure, and customizable AI deployments.  
- **Visual Fidelity Rising**: Advances like those suggested by the Nano Banana 2 leak point to a push for hyper-realistic AI-generated media that reduce manual editing needs—a potential game-changer for creative workflows.

---

In summary, generative AI continues to evolve rapidly—advancing in adaptability, domain applications, scientific discovery, creative capabilities, and enterprise utility. Let me know if you’d like to deep-dive into any of these developments—for instance, technical specifics of GPT‑5.1, the architecture behind Dragon Hatchling, or Marble’s integration with game engines.

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [36]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here’s an in-depth update on the *latest developments in generative AI* as of today, **November 15, 

### Continue Query with Web Search

In [37]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [38]:
display(Markdown(continue_search_response.output_text))

Here is a comprehensive roundup of **recent news** and developments in **generative AI**, covering a range of innovations, concerns, and industry shifts. All information is up-to-date as of **November 15, 2025**, with each statement carefully sourced.

---

Nearly all global creators (86%) now employ generative AI in their workflows, with 81% acknowledging it helps them produce content they otherwise could not. Creative applications include media editing (55%), asset generation (52%), and ideation (48%). Nevertheless, a majority (69%) are concerned about their content being used without consent. This snapshot comes from a recent Adobe survey released in early November 2025. ([techradar.com](https://www.techradar.com/pro/nearly-all-creators-admit-they-use-ai-tools-for-work-so-is-this-the-end-of-true-creativity?utm_source=openai))

---

###  Generative AI in Business: Ambition vs. Reality

A study by MIT, titled *"The GenAI Divide: State of AI in Business 2025"*, found that **95% of generative AI projects fail to deliver meaningful results**. Despite over **$44 billion** invested in startups and enterprise tools in the first half of 2025, most projects have not yielded notable productivity or revenue gains. The report attributes failures to inflated expectations, poor integration, and inadequate domain customization, warning of a looming “AI bubble.” ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/mit-study-finds-95-of-generative-ai-projects-are-failing-only-hype-little-transformation/articleshow/123453071.cms?utm_source=openai))

---

###  Enterprise Solutions: Adobe AI Foundry Launch

On **October 20, 2025**, Adobe unveiled **Adobe AI Foundry**, a new service that enables businesses to create customized generative AI models using their own media assets—spanning images, video, audio, and graphics. Brands like Home Depot and Disney Imagineering are early adopters. Following the announcement, Adobe’s stock rose 2.5% to **$341.55**, though it remains **down 25% year-to-date** due to stiff competition. ([investors.com](https://www.investors.com/news/technology/adobe-stock-adbe-tailored-gen-ai-models/?utm_source=openai))

---

###  Media & Entertainment: Netflix’s AI Integration

In **July 2025**, Netflix incorporated generative AI in one of its original series—**El Eternauta**. A visually complex scene of a building collapse in Buenos Aires was created using AI-based tools, completing it **ten times faster than traditional VFX methods**. Co-CEO Ted Sarandos emphasized that AI enhances storytelling and production efficiency, rather than replacing creators. ([theguardian.com](https://www.theguardian.com/media/2025/jul/18/netflix-uses-generative-ai-in-show-for-first-time-el-eternauta?utm_source=openai))

---

###  Emerging Model & Tech Highlights

- **Runway Gen‑4 (AI Video Model)**  
  Released in **March 2025**, this text-to-video tool generates up to 10-second clips in 720p resolution and 24 fps. It offers visual consistency within clips and incorporates features like camera movement effects and style transfer. However, consistency doesn’t extend over separate clips, and output length remains limited. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gen-4_%28AI_image_and_video_model%29?utm_source=openai))

- **Kling AI (China’s Text-to-Video Model)**  
  Developed by Kuaishou, Kling AI advanced to version **2.1 by May 2025**, offering 720p/1080p resolution, up to 30 fps, and improved realism with configurable modes. The model has also faced concerns over strict content censorship and misuse via malware from fake websites. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kling_AI?utm_source=openai))

- **GPT‑5.1 by OpenAI**  
  Released on **November 12, 2025**, GPT-5.1 brings enhancements over GPT‑5, including fewer hallucinations, better instruction-following, a "warmer" personality, and two operational modes: **Instant** (for quick answers) and **Thinking** (for deep reasoning). ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5.1?utm_source=openai))

- **Long-Context Multimodal Models**  
  Mid-2025 saw major AI models gain ultra-long context capabilities—Google’s Gemini 2.5 Pro and OpenAI’s GPT‑4.1 family now support up to **1 million tokens** and integrate multimodal inputs and improved reasoning. Meta also released **Llama 4**, a multi-modal model using a mixture-of-experts design for efficient performance. ([anvisai.com](https://anvisai.com/latest-breakthroughs-in-ai-5-key-innovations-june-2025/?utm_source=openai))

- **Startups & Open-Source Advances**  
  - **Reve Image** launched an image model noted for photorealism and superior prompt-text interpretation.  
  - **Ideogram 3.0** introduced enhanced multilingual capability, style referencing, and randomness in style generation.  
  - **Alibaba’s LHM** converts a single full-body photo into a 3D animated human avatar suitable for integration in games and virtual experiences. ([yukitaylor00.medium.com](https://yukitaylor00.medium.com/ai-weekly-top-10-ai-breakthroughs-you-missed-this-week-march-2025-1d742259e5b8?utm_source=openai))

- **Hardware & Regulation Trends**  
  AI regulation advanced significantly in 2025. The **EU AI Act** has gone into effect, establishing safety and transparency rules. The **US** launched a National AI Safety Board, while **China** imposed tighter audits on generative models. Meanwhile, hardware innovation accelerated with NVIDIA’s **Blackwell architecture** offering ~40% performance improvements, and Apple's **Neural Engine 2.0** enabling efficient on-device AI. ([bytenest.tech](https://bytenest.tech/latest-news-on-artificial-intelligence-in-2025-key-trends-breakthroughs-and-challenges/?utm_source=openai))

---

###  Summary Table

| Area                 | Key Highlights                                                                 |
|----------------------|--------------------------------------------------------------------------------|
| Creator Sentiment     | 86% use generative AI; 81% see creative gains; concerns over training usage     |
| Enterprise Reality   | 95% of projects fail; Adobe moves to bespoke models for brands                 |
| Media & VFX          | Netflix achieves 10× faster VFX via generative AI                              |
| Model Innovation     | GPT‑5.1, Runway Gen‑4, Kling AI, long-context multimodal models, open-source     |
| Regulations & Hardware | EU AI Act effective; US safety board; hardware boosts from NVIDIA and Apple   |

---

In summary, while generative AI continues to **expand creatively and technologically**—from personalized enterprise tools to groundbreaking model capabilities—it also faces challenges like project failure, ethical concerns, and regulatory hurdles. If you'd like, I can explore any of these developments further—just let me know which interests you most!

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [39]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [40]:
display(Markdown(combined_search_response.output_text))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of **November 15, 2025**, with detailed citations and structured analysis:

---

##  Major Model Releases and Innovations

- **OpenAI GPT‑5.1**  
  Released on **November 12, 2025**, GPT‑5.1 is the newest iteration in OpenAI’s GPT series. It introduces eight selectable personality options, improved instruction-following, reduced hallucinations, and two operational modes: **Instant** (for speed) and **Thinking** (for complex reasoning tasks) ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5.1?utm_source=openai)).

- **OpenAI GPT‑5**  
  Launched earlier on **August 7, 2025**, GPT‑5 unified reasoning and multimodal capabilities under a single interface and was integrated into ChatGPT and Microsoft Copilot ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai)).

- **OpenAI o4‑mini**  
  Released on **April 16, 2025**, this compact reasoning model supports both text and image inputs, including whiteboard sketch analysis. A higher-accuracy variant, **o4‑mini‑high**, is available to paid-tier users ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_o4-mini?utm_source=openai)).

- **Google Gemini 2.5 Pro & Flash**  
  Google’s Gemini 2.5 Pro (with “Deep Think” reasoning mode) and Flash models became generally available on **June 17, 2025**. These models offer enhanced reasoning, coding capabilities, native audio output, and a massive 1 million token context window ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai)).

- **Nano Banana (Gemini 2.5 Flash Image)**  
  Launched on **August 26, 2025**, this image generation and editing model went viral for its photorealistic “3D figurine” outputs. It supports features like subject consistency, multi-image fusion, and SynthID watermarking. Within weeks, it attracted over 10 million new users and facilitated more than 200 million image edits ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai)).

- **Other Notable Models**  
  - **GPT‑4.5 (Orion)**: Released in **February 2025**, it offers reduced hallucinations, improved accuracy, and a large context window, excelling in creative and emotionally intelligent tasks ([medium.com](https://medium.com/%4017shubhanshujain/top-generative-ai-trends-for-2025-gpt-4-5-gemini-2-5-and-ethical-implications-94a126df84d8?utm_source=openai)).  
  - **Llama 4 Family**: Released by Meta on **April 5, 2025**, these open-source models support early-fusion multimodality, MoE architecture, and extensive language coverage ([medium.com](https://medium.com/%4017shubhanshujain/top-generative-ai-trends-for-2025-gpt-4-5-gemini-2-5-and-ethical-implications-94a126df84d8?utm_source=openai)).  
  - **Baidu Ernie 4.5 & Ernie X1**: Announced in **April 2025**, these multimodal and reasoning models reportedly outperform competitors on benchmarks. Ernie 4.5 is slated to become open-source from **June 30, 2025** ([globenewswire.com](https://www.globenewswire.com/news-release/2025/04/17/3063915/0/en/Applied-Generative-AI-Course-Launched-by-Interview-Kickstart-2025-Best-GenAI-Course-With-Agentic-AI-Projects-For-Top-AI-Jobs-at-Google-Meta-Netflix-Microsoft-OpenAI-Nvidia.html?utm_source=openai)).

---

##  Emerging Trends and Market Dynamics

- **Agentic AI and Autonomous Agents**  
  The shift from AI as copilots to fully autonomous agents is accelerating. Agentic AI systems can plan, act, and adapt independently—expected to be a dominant trend in 2026 and beyond ([simplilearn.com](https://www.simplilearn.com/top-technology-trends-and-jobs-article?utm_source=openai)).

- **Generative AI Market Growth**  
  The generative AI market experienced triple-digit growth across hardware, foundation models, and development platforms in 2024. Forecasts estimate over **US$400 billion** in AI-related spending in 2025 ([businesswire.com](https://www.businesswire.com/news/home/20250825682581/en/Generative-AI-Market-Report-2025-GenAI-Market-Experienced-Triple-digit-growth-Rates-in-All-Three-Major-Segments-Spanning-GenAI-Hardware-Foundation-Models-and-Development-Platforms---ResearchAndMarkets.com?utm_source=openai)).

- **Enterprise Adoption and Scaling**  
  According to McKinsey’s 2025 Global AI Survey, about one-third of organizations are scaling AI programs, while 23% are deploying agentic AI systems. High performers are investing heavily—over 20% of digital budgets—into AI and redesigning workflows to capture value ([mckinsey.com](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai?utm_source=openai)).

- **Responsible AI Practices**  
  UC Berkeley released a “Responsible Use of Generative AI” playbook outlining ten actionable strategies for business leaders and product managers to ensure ethical and responsible deployment of GenAI ([weforum.org](https://www.weforum.org/stories/2025/06/responsible-generative-ai-product-development-use/?utm_source=openai)).

- **Generative AI in Software Development**  
  Bain’s September 2025 report highlights that while GenAI tools boost productivity, organizations must adapt processes to realize real value from these technologies ([bain.com](https://www.bain.com/insights/from-pilots-to-payoff-generative-ai-in-software-development-technology-report-2025/?utm_source=openai)).

- **Hardware Innovations**  
  Qualcomm unveiled the **AI200** and **AI250** inference accelerators, targeting data center deployment in 2026 and 2027. These systems emphasize scalability, efficiency, and compatibility with major AI frameworks ([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai)).

---

##  Consumer-Facing and Multimedia Advances

- **OpenAI Sora 2 and Sora App**  
  On **September 30, 2025**, OpenAI launched **Sora 2**, a multimodal video+audio generation model, alongside the **Sora app**—a social-style platform for short AI-generated clips. The launch raised significant discussions around IP, moderation, and deepfake risks ([champaignmagazine.com](https://champaignmagazine.com/2025/10/05/ai-by-ai-weekly-top-5-september-29-october-5-2025/?utm_source=openai)).

- **Google Veo 3.1**  
  Released in **October 2025**, Veo 3.1 enhances narrative control in generative video, enabling seamless transitions, character consistency, and integrated audio. It’s available via Gemini API and Google’s Flow editing tool ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai)).

- **Anthropic Claude Haiku 4.5**  
  Also launched in **October 2025**, this compact model delivers flagship-level reasoning and coding performance at significantly higher speed and lower cost. It supports agentic capabilities like “using computers” and process identification ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai)).

---

##  Academic and Research Developments

- **Chronologically Consistent Generative AI**  
  A recent October 2025 study introduced models trained only on data available before a defined cutoff, eliminating lookahead bias. These models offer replicability and conservative forecast accuracy, useful for prediction tasks ([arxiv.org](https://arxiv.org/abs/2510.11677?utm_source=openai)).

- **Agentic AI Networking in 6G**  
  A March 2025 research paper proposed **AgentNet**, a framework for collaborative, embodied AI agents in networking environments. It envisions generative foundation models acting as agents in industrial automation and metaverse infotainment systems ([arxiv.org](https://arxiv.org/abs/2503.15764?utm_source=openai)).

---

##  Summary Table

| Category                  | Key Developments                                                                 |
|---------------------------|----------------------------------------------------------------------------------|
| Model Releases            | GPT‑5.1, GPT‑5, o4‑mini, Gemini 2.5 Pro/Flash, Nano Banana, GPT‑4.5, Llama 4, Ernie 4.5/X1 |
| Market & Enterprise Trends| Agentic AI, market growth, enterprise scaling, responsible AI, hardware advances |
| Consumer & Multimedia     | Sora 2, Veo 3.1, Claude Haiku 4.5                                               |
| Research & Academia       | Chronologically consistent models, AgentNet for 6G networks                      |

---

###  Final Thoughts

The generative AI landscape in late 2025 is defined by rapid innovation across models, enterprise adoption, hardware, and ethical frameworks. Notably:

- **GPT‑5.1** (Nov 12) and **Nano Banana** (Aug 26) are among the most recent high-impact releases.
- **Agentic AI** is emerging as a transformative trend, with enterprises beginning to scale such systems.
- **Responsible AI** and **hardware infrastructure** are becoming critical enablers of sustainable and scalable AI deployment.
- **Consumer-facing tools** like Sora 2 and Veo 3.1 are pushing generative AI into mainstream creative workflows.

Let me know if you'd like a deeper dive into any specific model, trend, or application area!

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [25]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_6917fca1950c8191936b987c0b117850


In [41]:
with open('sports_notes.txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-6bcYuAv1fiyC1mW6oj9qFN


In [42]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-6bcYuAv1fiyC1mW6oj9qFN


In [47]:
query = "Explain the rules of basketball"

In [49]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

Basketball relies heavily on teamwork, spacing, and fast transitions from defense to offense.
Soccer
 Relevant score: 0.15570516833258916
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1e1
 Relevant score: 0.006318813056261807
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1f1
 Relevant score: 0.004876320483920653
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1d1
 Relevant score: 0.003997826679217775
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1f1
 Relevant score: 0.0035837128264293674


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [50]:
file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [51]:
display(Markdown(file_search_response.output_text))

Basketball is a team sport with specific rules that govern gameplay. Here are the basic rules:

1. **Objective**: The main goal is to score points by shooting the basketball through the opponent's hoop.

2. **Teams**: Each team consists of five players on the court at a time.

3. **Game Duration**: A standard game is divided into four quarters, each lasting 12 minutes in the NBA. Other leagues may have different durations.

4. **Scoring**: Points are scored as follows:
   - Field goals are worth two points, or three points if the shooter is behind the three-point line.
   - Free throws are worth one point each.

5. **Dribbling**: Players must dribble the ball (bounce it on the floor) while moving. If a player stops dribbling, they cannot start again until another player touches the ball.

6. **Violations**: Common violations include traveling (taking too many steps without dribbling), double dribble (dribbling with both hands or starting to dribble again after stopping), and shot clock violations (failing to attempt a shot within the allotted time).

7. **Fouls**: Physical contact that impedes an opponent can result in a foul. Accumulating too many fouls can lead to free throws for the opposing team.

8. **Possession**: The game starts with a jump ball, and possession alternates between teams after each quarter.

9. **Defense**: Teams can play man-to-man or zone defense to prevent the opposing team from scoring.

10. **Substitutions**: Players can be substituted in and out of the game during stoppages in play.

These rules ensure fair play and structure within the game, allowing for competitive and strategic gameplay.

In [52]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [53]:
display(Markdown(file_search_response.output_text))

Basketball is a team sport with specific rules that govern gameplay. Here are the basic rules:

1. **Objective**: The main goal is to score points by shooting the basketball through the opponent's hoop.

2. **Teams**: Each team consists of five players on the court at a time.

3. **Game Duration**: A standard game is divided into four quarters, each lasting 12 minutes in the NBA. Other leagues may have different durations.

4. **Scoring**: Points are scored as follows:
   - Field goals are worth two points, or three points if the shooter is behind the three-point line.
   - Free throws are worth one point each.

5. **Dribbling**: Players must dribble the ball (bounce it on the floor) while moving. If a player stops dribbling, they cannot start again until another player touches the ball.

6. **Violations**: Common violations include traveling (taking too many steps without dribbling), double dribble (dribbling with both hands or starting to dribble again after stopping), and shot clock violations (failing to attempt a shot within the allotted time).

7. **Fouls**: Physical contact that impedes an opponent can result in a foul. Accumulating too many fouls can lead to free throws for the opposing team.

8. **Possession**: The game starts with a jump ball, and possession alternates between teams after each quarter.

9. **Defense**: Teams can play man-to-man or zone defense to prevent the opposing team from scoring.

10. **Substitutions**: Players can be substituted in and out of the game during stoppages in play.

These rules ensure fair play and structure within the game, allowing for competitive and strategic gameplay.

In [55]:
continue_query = '“Summarize the different types of t.”'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [56]:
display(Markdown(continue_search_response.output_text))

It seems you mentioned uploading notes, but I don't have access to external files. If you provide key points or text from your notes, I can help incorporate them with the latest information from the web. Let me know how you'd like to proceed!